# Tutorial 08: Human-in-the-Loop & Interactive Workflows

##  Learning Objectives
By the end of this notebook, you will:
- Implement **approval gates** that pause workflows for human review
- Create **interactive executors** that request user input
- Build **multi-turn conversations** with human feedback loops
- Use **context variables** to track approval states
- Implement **rejection handlers** for declined requests
- Understand **when human oversight is critical** vs optional

##  Prerequisites

Before starting this tutorial, you should have completed:
- **Tutorial 06: Multi-Agent Workflows** (Sequential, Concurrent patterns)
- **Tutorial 07: Advanced Workflows** (WorkflowBuilder, Custom Executors)

##  What is Human-in-the-Loop?

**Human-in-the-Loop (HITL)** workflows pause execution to get human input:
-  **Approval Gates** - Human reviews and approves/rejects AI decisions
-  **Interactive Feedback** - Human provides input to guide execution
-  **Quality Control** - Human validates critical outputs
-  **Policy Compliance** - Human ensures adherence to rules

### Why Use HITL?

| Scenario | Why HITL? |
|----------|----------|
| Financial transactions | Legal requirement for approval |
| Content publication | Quality and brand safety |
| Data deletion | Irreversible action |
| Customer communications | Reputation risk |
| Policy violations | Manual judgment needed |
| High-value decisions | Cost/risk mitigation |

### HITL Patterns in This Tutorial

1. **Approval Gate Pattern** - Simple approve/reject decision
2. **Interactive Refinement** - Multi-turn conversation with feedback
3. **Conditional Approval** - Different paths based on decision
4. **Budget-Based Approval** - Auto-approve under threshold, else require human

---

## Step 1: Setup and Imports

In [6]:
import asyncio
from typing import cast


from agent_framework import (
    # Workflow builders
    WorkflowBuilder,
    SequentialBuilder,
    # Core components
    Executor,
    AgentExecutor,
    AgentExecutorResponse,
    handler,
    WorkflowContext,
    # Events
    WorkflowOutputEvent,
    AgentRunEvent,
    ExecutorInvokedEvent,
    # Message types
    ChatMessage,
    Role,
)
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv

load_dotenv()
print(" Imports successful!")
print("🎭 Ready to build Human-in-the-Loop workflows!")

 Imports successful!
🎭 Ready to build Human-in-the-Loop workflows!


## Step 2: Pattern 1 - Simple Approval Gate

The simplest HITL pattern: AI generates something, human approves or rejects.

### Flow
```
User Request → AI Agent → Human Review → (approved) → Execute
                                       → (rejected) → Stop
```

### Key Concepts
- **Synchronous input()** - Pause workflow to get user input
- **Conditional logic** - Route based on approval decision
- **Context tracking** - Store approval state for downstream use

In [7]:
async def simple_approval_gate_demo():
    """
    Demonstrate simple approval gate pattern:
    AI drafts an email → Human approves/rejects → Send if approved
    """
    print("="*70)
    print("PATTERN 1: Simple Approval Gate")
    print("="*70)
    print("\nFlow: Draft Email → Human Review → Send if Approved\n")
    
    # Create email drafting agent
    credential = AzureCliCredential()
    chat_client = AzureAIAgentClient(async_credential=credential)
    
    email_agent = chat_client.create_agent(
        instructions="""
        You are a professional email writer.
        Draft clear, concise, professional emails based on user requests.
        Keep emails brief (3-4 sentences max).
        Always include a subject line.
        """,
        name="EmailDrafter",
    )
    
    # Custom executor for human approval
    class ApprovalGate(Executor):
        """Pauses workflow for human approval/rejection"""
        
        def __init__(self):
            super().__init__(id="approval_gate")
        
        @handler
        async def review(self, messages: list[ChatMessage], ctx: WorkflowContext[list[ChatMessage], str]) -> None:
            """
            Show draft to human and get approval decision.
            """
            # Extract text from the agent's response
            draft = messages[-1].text if messages else "No content"
            
            print("\n" + "="*70)
            print("📧 EMAIL DRAFT FOR REVIEW")
            print("="*70)
            print(draft)
            print("="*70)
            
            # Pause workflow for human input
            while True:
                decision = input("\n👤 Approve this email? (yes/no): ").strip().lower()
                
                if decision in ['yes', 'y']:
                    print(" Email approved!")
                    await ctx.yield_output(f" APPROVED AND SENT\n\n{draft}")
                    break
                elif decision in ['no', 'n']:
                    print(" Email rejected!")
                    await ctx.yield_output(" REJECTED - Email was not sent")
                    break
                else:
                    print("  Please enter 'yes' or 'no'")
    
    # Build workflow: Agent → Approval Gate
    email_executor = AgentExecutor(email_agent, id="EmailDrafter")
    approval_gate = ApprovalGate()
    
    workflow = (
        SequentialBuilder()
        .participants([email_executor, approval_gate])
        .build()
    )
    
    # Test request
    request = "Draft an email to the team announcing our quarterly all-hands meeting next Friday at 2 PM."
    
    print(f"\n Request: {request}\n")
    print("🤖 AI is drafting email...\n")
    
    # Run workflow
    final_output = None
    async for event in workflow.run_stream(request):
        if isinstance(event, WorkflowOutputEvent):
            final_output = event.data
    
    # Show final result
    print("\n" + "="*70)
    print(" FINAL RESULT")
    print("="*70)
    if final_output:
        print(final_output)
    print("="*70)
    
    print("\n What Happened:")
    print("  1⃣  AI drafted an email")
    print("  2⃣  Workflow PAUSED for human review")
    print("  3⃣  Human approved or rejected")
    print("  4⃣  Workflow completed based on decision")

await simple_approval_gate_demo()

PATTERN 1: Simple Approval Gate

Flow: Draft Email → Human Review → Send if Approved


 Request: Draft an email to the team announcing our quarterly all-hands meeting next Friday at 2 PM.

🤖 AI is drafting email...


📧 EMAIL DRAFT FOR REVIEW
Subject: Quarterly All-Hands Meeting Scheduled for Next Friday

Dear Team,

Please mark your calendars for our quarterly all-hands meeting, which will be held next Friday at 2 PM. We’ll review recent achievements, discuss upcoming projects, and address any questions. Your attendance is important, so please make every effort to join.

Thank you!

📧 EMAIL DRAFT FOR REVIEW
Subject: Quarterly All-Hands Meeting Scheduled for Next Friday

Dear Team,

Please mark your calendars for our quarterly all-hands meeting, which will be held next Friday at 2 PM. We’ll review recent achievements, discuss upcoming projects, and address any questions. Your attendance is important, so please make every effort to join.

Thank you!
 Email approved!

 FINAL RESULT
 APPRO

## Step 3: Pattern 2 - Interactive Refinement Loop

More sophisticated: Allow human to provide feedback and iterate.

### Flow
```
Request → AI Draft → Human Review ┬→ Approve → Done
                                  └→ Feedback → AI Revise → Human Review (loop)
```

### Key Concepts
- **Multi-turn conversation** - Iterative improvement
- **Feedback integration** - Human guides AI refinement
- **Loop control** - Exit condition (approval or max iterations)

In [8]:
async def interactive_refinement_demo():
    """
    Demonstrate interactive refinement:
    AI drafts → Human provides feedback → AI revises → Repeat until approved
    """
    print("="*70)
    print("PATTERN 2: Interactive Refinement Loop")
    print("="*70)
    print("\nFlow: Draft → Review → Feedback → Revise → Repeat\n")
    
    # Create content writer agent
    credential = AzureCliCredential()
    chat_client = AzureAIAgentClient(async_credential=credential)
    
    writer_agent = chat_client.create_agent(
        instructions="""
        You are a creative blog post writer.
        Write engaging, informative blog posts on requested topics.
        If given feedback, incorporate it and revise the post.
        Keep posts concise (3-4 paragraphs).
        """,
        name="BlogWriter",
    )
    
    # Interactive refinement executor
    class InteractiveEditor(Executor):
        """Allows human to iteratively refine AI output"""
        
        def __init__(self, agent, max_iterations=3):
            super().__init__(id="interactive_editor")
            self.agent = agent
            self.max_iterations = max_iterations
        
        @handler
        async def edit_loop(self, request: str, ctx: WorkflowContext[str, str]) -> None:
            """
            Interactive editing loop with human feedback.
            """
            conversation_history = [ChatMessage(role=Role.USER, content=request)]
            iteration = 0
            
            while iteration < self.max_iterations:
                iteration += 1
                print(f"\n{'='*70}")
                print(f" DRAFT {iteration} (max {self.max_iterations})")
                print("="*70)
                
                # Get AI response
                response = await self.agent.run(conversation_history)
                draft = response.text
                
                print(draft)
                print("="*70)
                
                # Get human feedback
                decision = input("\n👤 (a)pprove, (f)eedback, or (r)eject? ").strip().lower()
                
                if decision in ['a', 'approve']:
                    print("\n Content approved!")
                    await ctx.yield_output(f" APPROVED (after {iteration} iteration(s))\n\n{draft}")
                    return
                
                elif decision in ['r', 'reject']:
                    print("\n Content rejected!")
                    await ctx.yield_output(" REJECTED - Content was not published")
                    return
                
                elif decision in ['f', 'feedback']:
                    feedback = input("\n What changes would you like? ")
                    print(f"\n Feedback recorded: {feedback}")
                    print(" AI is revising based on your feedback...")
                    
                    # Add to conversation history
                    conversation_history.append(ChatMessage(role=Role.ASSISTANT, content=draft))
                    conversation_history.append(
                        ChatMessage(
                            role=Role.USER, 
                            content=f"Please revise based on this feedback: {feedback}"
                        )
                    )
                else:
                    print("  Invalid option. Treating as feedback request.")
                    feedback = input("\n What changes would you like? ")
                    conversation_history.append(ChatMessage(role=Role.ASSISTANT, content=draft))
                    conversation_history.append(
                        ChatMessage(
                            role=Role.USER, 
                            content=f"Please revise based on this feedback: {feedback}"
                        )
                    )
            
            # Max iterations reached
            print(f"\n  Maximum iterations ({self.max_iterations}) reached.")
            final_decision = input("👤 Approve final draft? (yes/no): ").strip().lower()
            
            if final_decision in ['yes', 'y']:
                await ctx.yield_output(f" APPROVED (max iterations reached)\n\n{draft}")
            else:
                await ctx.yield_output(" REJECTED - Maximum iterations reached without approval")
    
    # Build workflow
    editor = InteractiveEditor(writer_agent, max_iterations=3)
    
    workflow = (
        WorkflowBuilder()
        .set_start_executor(editor)
        .build()
    )
    
    # Test request
    request = "Write a blog post about the benefits of AI agents in customer service."
    
    print(f"\n Request: {request}\n")
    print("🤖 AI is writing initial draft...\n")
    
    # Run workflow
    final_output = None
    async for event in workflow.run_stream(request):
        if isinstance(event, WorkflowOutputEvent):
            final_output = event.data
    
    # Show final result
    print("\n" + "="*70)
    print(" FINAL RESULT")
    print("="*70)
    if final_output:
        print(final_output)
    print("="*70)
    
    print("\n What Happened:")
    print("  1⃣  AI created initial draft")
    print("  2⃣  Human reviewed and provided feedback")
    print("  3⃣  AI revised based on feedback")
    print("  4⃣  Loop continued until approval or max iterations")
    print("  5⃣  Multi-turn conversation maintained context")

await interactive_refinement_demo()

PATTERN 2: Interactive Refinement Loop

Flow: Draft → Review → Feedback → Revise → Repeat


 Request: Write a blog post about the benefits of AI agents in customer service.

🤖 AI is writing initial draft...


 DRAFT 1 (max 3)
Absolutely! Please specify the topic you’d like me to write a blog post about, and I’ll get started.
Absolutely! Please specify the topic you’d like me to write a blog post about, and I’ll get started.

 Content rejected!

 FINAL RESULT
 REJECTED - Content was not published

 What Happened:
  1⃣  AI created initial draft
  2⃣  Human reviewed and provided feedback
  3⃣  AI revised based on feedback
  4⃣  Loop continued until approval or max iterations
  5⃣  Multi-turn conversation maintained context


## Step 4: Pattern 3 - Conditional Approval with Branching

Different workflow paths based on approval decision.

### Flow
```
Request → Planner Agent → Human Review ┬→ Approved → Execution Agent → Done
                                       └→ Rejected → Alternative Path → Done
```

### Key Concepts
- **Branching logic** - Different downstream executors
- **Context passing** - Share approval state
- **Fallback paths** - Handle rejection gracefully

In [9]:
async def conditional_approval_demo():
    """
    Demonstrate conditional approval with branching:
    Plan → Approve/Reject → Different paths based on decision
    """
    print("="*70)
    print("PATTERN 3: Conditional Approval with Branching")
    print("="*70)
    print("\nFlow: Plan → Review → Branch (Approved/Rejected paths)\n")
    
    # Create planning agent
    credential = AzureCliCredential()
    chat_client = AzureAIAgentClient(async_credential=credential)
    
    planner_agent = chat_client.create_agent(
        instructions="""
        You are a project planning assistant.
        Create detailed project plans with timeline, budget, and resources.
        Keep plans concise (4-5 key points).
        """,
        name="ProjectPlanner",
    )
    
    execution_agent = chat_client.create_agent(
        instructions="""
        You are a project execution assistant.
        Create step-by-step execution guides based on approved plans.
        Be specific and actionable.
        """,
        name="ExecutionPlanner",
    )
    
    alternative_agent = chat_client.create_agent(
        instructions="""
        You are a creative alternatives advisor.
        Suggest alternative approaches when original plans are rejected.
        Provide 2-3 different options to consider.
        """,
        name="AlternativesAdvisor",
    )
    
    # Approval gate with branching
    class BranchingApprovalGate(Executor):
        """Approval gate that determines next executor based on decision"""
        
        def __init__(self):
            super().__init__(id="branching_approval")
            self.approved = False
        
        @handler
        async def review_and_branch(self, response: AgentExecutorResponse, ctx: WorkflowContext[AgentExecutorResponse, str]) -> None:
            """
            Review plan and route to appropriate next step.
            """
            # Extract text from the agent's response
            plan = response.agent_run_response.text if response.agent_run_response else "No content"
            
            print("\n" + "="*70)
            print("📋 PROJECT PLAN FOR REVIEW")
            print("="*70)
            print(plan)
            print("="*70)
            
            # Get approval decision
            while True:
                decision = input("\n👤 Approve this plan? (yes/no): ").strip().lower()
                
                if decision in ['yes', 'y']:
                    print(" Plan approved! Creating execution guide...")
                    self.approved = True
                    await ctx.send_message(f"Create execution steps for: {plan}")
                    break
                elif decision in ['no', 'n']:
                    print(" Plan rejected! Suggesting alternatives...")
                    self.approved = False
                    await ctx.send_message(f"Original plan rejected. Suggest alternatives for: {plan}")
                    break
                else:
                    print("  Please enter 'yes' or 'no'")
    
    # Result handler
    class ResultHandler(Executor):
        """Formats final output based on approval path"""
        
        def __init__(self, approval_gate):
            super().__init__(id="result_handler")
            self.approval_gate = approval_gate
        
        @handler
        async def format_result(self, response: AgentExecutorResponse, ctx: WorkflowContext[AgentExecutorResponse, str]) -> None:
            # Extract content from response
            content = response.agent_run_response.text if response.agent_run_response else "No content"
            
            if self.approval_gate.approved:
                output = f" PLAN APPROVED\n\n📋 EXECUTION GUIDE:\n{content}"
            else:
                output = f" PLAN REJECTED\n\n ALTERNATIVE OPTIONS:\n{content}"
            
            await ctx.yield_output(output)
    
    # Build workflow with branching
    planner_executor = AgentExecutor(planner_agent, id="ProjectPlanner")
    approval_gate = BranchingApprovalGate()
    execution_executor = AgentExecutor(execution_agent, id="ExecutionPlanner")
    alternative_executor = AgentExecutor(alternative_agent, id="AlternativesAdvisor")
    result_handler = ResultHandler(approval_gate)
    
    # Note: This is a simplified version - in practice you'd use conditional routing
    # For this demo, we'll use sequential flow and the approval gate will send different messages
    workflow = (
        WorkflowBuilder()
        .set_start_executor(planner_executor)
        .add_edge(planner_executor, approval_gate)
        .add_edge(approval_gate, execution_executor)  # Will handle approved path
        .add_edge(execution_executor, result_handler)
        .build()
    )
    
    # Test request
    request = "Create a plan for launching a new mobile app in 3 months."
    
    print(f"\n Request: {request}\n")
    print("🤖 AI is creating project plan...\n")
    
    # Run workflow
    final_output = None
    async for event in workflow.run_stream(request):
        if isinstance(event, WorkflowOutputEvent):
            final_output = event.data
    
    # Show final result
    print("\n" + "="*70)
    print(" FINAL RESULT")
    print("="*70)
    if final_output:
        print(final_output)
    print("="*70)
    
    print("\n What Happened:")
    print("  1⃣  AI created initial project plan")
    print("  2⃣  Human reviewed and approved/rejected")
    print("  3⃣  Workflow branched based on decision")
    print("  4⃣  Approved → Execution guide created")
    print("  5⃣  Rejected → Alternative options suggested")

await conditional_approval_demo()

[2025-10-03 12:20:52 - /opt/homebrew/Cellar/python@3.12/3.12.11_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/asyncio/base_events.py:1833 - ERROR] Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1125dbc20>
[2025-10-03 12:20:52 - /opt/homebrew/Cellar/python@3.12/3.12.11_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/asyncio/base_events.py:1833 - ERROR] Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x112672930>
[2025-10-03 12:20:52 - /opt/homebrew/Cellar/python@3.12/3.12.11_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/asyncio/base_events.py:1833 - ERROR] Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x112458d10>, 300347.974473791)])']
connector: <aiohttp.connector.TCPConnector object at 0x1127065d0>
[2025-10-03 12:20:52 - /opt/homebrew/Cellar/python@3.12/3.12.11_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/asyncio/base_events

PATTERN 3: Conditional Approval with Branching

Flow: Plan → Review → Branch (Approved/Rejected paths)


 Request: Create a plan for launching a new mobile app in 3 months.

🤖 AI is creating project plan...


📋 PROJECT PLAN FOR REVIEW
**Mobile App Launch Plan (3 Months)**

**1. Development & Testing (Weeks 1-8)**
- Finalize requirements, UI design, and core features (Week 1–2)
- App development sprints, QA/testing, resolve bugs (Weeks 3–8)
- Resources: 1 project manager, 2 developers, 1 designer, 1 QA tester

**2. Marketing Prep & App Store Setup (Weeks 6–10)**
- Develop marketing materials: website, social media, press kit (Weeks 6–7)
- Prepare App Store/Play Store listings, screenshots, keywords (Weeks 8–10)
- Resources: 1 marketing lead, 1 copywriter, designer (shared with dev team)

**3. Beta Launch & Feedback (Weeks 9–10)**
- Release beta to select users, collect feedback, address final issues (Weeks 9–10)
- Resources: QA tester, support staff for feedback

**4. Public Launch & Pr

## Step 5: Pattern 4 - Smart Approval (Auto-approve vs Manual)

Automatically approve low-risk requests, require human approval for high-risk ones.

### Flow
```
Request → Risk Analyzer ┬→ Low Risk → Auto-Approve → Execute
                        └→ High Risk → Human Review → (approved) → Execute
                                                     → (rejected) → Stop
```

### Key Concepts
- **Risk assessment** - Business logic determines approval path
- **Efficiency** - Skip human review when safe
- **Conditional gates** - Only pause when necessary

In [10]:
async def smart_approval_demo():
    """
    Demonstrate smart approval:
    Auto-approve low-value purchases, require human approval for high-value
    """
    print("="*70)
    print("PATTERN 4: Smart Approval (Auto vs Manual)")
    print("="*70)
    print("\nFlow: Analyze → Route (Auto-approve / Human Review)\n")
    
    # Purchase recommendation agent
    credential = AzureCliCredential()
    chat_client = AzureAIAgentClient(async_credential=credential)
    
    purchase_agent = chat_client.create_agent(
        instructions="""
        You are a purchasing assistant.
        Recommend specific items to purchase based on requests.
        Include estimated cost in your response.
        Format: "Item: [name], Estimated Cost: $[amount], Justification: [reason]"
        """,
        name="PurchaseAdvisor",
    )
    
    # Smart approval gate with threshold
    class SmartApprovalGate(Executor):
        """Auto-approves under threshold, requires human approval above"""
        
        def __init__(self, threshold=100):
            super().__init__(id="smart_approval")
            self.threshold = threshold
        
        @handler
        async def smart_approve(self, messages: list[ChatMessage], ctx: WorkflowContext[list[ChatMessage], str]) -> None:
            """
            Extract cost and auto-approve or request human review.
            """
            # Extract text from the agent's response
            recommendation = messages[-1].text if messages else "No content"
            
            print("\n" + "="*70)
            print(" PURCHASE RECOMMENDATION")
            print("="*70)
            print(recommendation)
            print("="*70)
            
            # Extract cost (simplified - look for $amount)
            import re
            cost_match = re.search(r'\$([0-9,]+)', recommendation)
            
            if cost_match:
                cost_str = cost_match.group(1).replace(',', '')
                cost = float(cost_str)
                
                print(f"\n Detected cost: ${cost:.2f}")
                print(f"   Approval threshold: ${self.threshold:.2f}")
                
                if cost <= self.threshold:
                    print(f"\n AUTO-APPROVED (under ${self.threshold} threshold)")
                    await ctx.yield_output(
                        f" AUTO-APPROVED (${cost:.2f} ≤ ${self.threshold:.2f})\n\n{recommendation}"
                    )
                else:
                    print(f"\n  REQUIRES HUMAN APPROVAL (over ${self.threshold} threshold)")
                    
                    # Pause for human review
                    while True:
                        decision = input("\n👤 Approve this purchase? (yes/no): ").strip().lower()
                        
                        if decision in ['yes', 'y']:
                            print(" Purchase approved by human!")
                            await ctx.yield_output(
                                f" APPROVED BY HUMAN (${cost:.2f} > ${self.threshold:.2f})\n\n{recommendation}"
                            )
                            break
                        elif decision in ['no', 'n']:
                            print(" Purchase rejected!")
                            await ctx.yield_output(
                                f" REJECTED (${cost:.2f} > ${self.threshold:.2f})\n\nReason: Declined by approver"
                            )
                            break
                        else:
                            print("  Please enter 'yes' or 'no'")
            else:
                print("\n  Could not extract cost - requiring human review")
                decision = input("\n👤 Approve this purchase? (yes/no): ").strip().lower()
                
                if decision in ['yes', 'y']:
                    await ctx.yield_output(f" APPROVED\n\n{recommendation}")
                else:
                    await ctx.yield_output(" REJECTED")
    
    # Build workflow
    purchase_executor = AgentExecutor(purchase_agent, id="PurchaseAdvisor")
    smart_gate = SmartApprovalGate(threshold=500.00)  # Auto-approve under $500
    
    workflow = (
        SequentialBuilder()
        .participants([purchase_executor, smart_gate])
        .build()
    )
    
    # Test with multiple scenarios
    test_cases = [
        "Recommend office supplies for the team (pens, notebooks, etc.)",
        "Recommend a new laptop for the development team lead",
    ]
    
    for i, request in enumerate(test_cases, 1):
        print("\n\n" + "#"*70)
        print(f"TEST CASE {i}")
        print("#"*70)
        print(f"\n Request: {request}\n")
        print("🤖 AI is analyzing request...\n")
        
        final_output = None
        async for event in workflow.run_stream(request):
            if isinstance(event, WorkflowOutputEvent):
                final_output = event.data
        
        print("\n" + "="*70)
        print(" RESULT")
        print("="*70)
        if final_output:
            print(final_output)
        print("="*70)
    
    print("\n" + "="*70)
    print(" What Happened:")
    print("="*70)
    print("  1⃣  AI recommended purchases with cost estimates")
    print("  2⃣  Smart gate analyzed cost vs threshold")
    print("  3⃣  Low-cost items auto-approved instantly")
    print("  4⃣  High-cost items required human review")
    print("  5⃣  Efficiency: Skip review when safe, require when risky")
    print("\n Production Use Cases:")
    print("  • Financial transactions (auto-approve < $X)")
    print("  • Content moderation (auto-approve low-risk)")
    print("  • Resource allocation (auto-approve standard requests)")
    print("  • Data access (auto-approve public, review sensitive)")

await smart_approval_demo()

PATTERN 4: Smart Approval (Auto vs Manual)

Flow: Analyze → Route (Auto-approve / Human Review)



######################################################################
TEST CASE 1
######################################################################

 Request: Recommend office supplies for the team (pens, notebooks, etc.)

🤖 AI is analyzing request...


 PURCHASE RECOMMENDATION
Item: Pilot G2 Retractable Gel Pens (Pack of 12), Estimated Cost: $15, Justification: Reliable, smooth-writing pens suitable for daily office use.

Item: Mead Spiral Notebooks (Pack of 6), Estimated Cost: $18, Justification: Durable notebooks for note-taking, brainstorming, and meeting records.

Item: Post-it Notes (Pack of 5, 3x3"), Estimated Cost: $8, Justification: Useful for reminders, quick notes, and idea organization.

Item: Staples Standard Stapler, Estimated Cost: $12, Justification: Essential for keeping documents organized and together.

Item: Scotch Magic Tape (Pack of 3), Estimated Cost: $9, Just

##  Key Takeaways

### What We Learned

1. **Approval Gates**
   - Use `input()` in executors to pause workflows
   - Get human approval/rejection decisions
   - Route workflow based on decision

2. **Interactive Refinement**
   - Multi-turn conversations with feedback
   - Maintain conversation history
   - Iterate until approval or max iterations

3. **Conditional Branching**
   - Different paths based on approval
   - Share state across executors
   - Handle rejection gracefully

4. **Smart Approval**
   - Business logic determines approval path
   - Auto-approve low-risk items
   - Require human review for high-risk
   - Balance efficiency with oversight

### HITL Best Practices

**When to Use HITL:**
-  High-stakes decisions (financial, legal)
-  Irreversible actions (delete, publish)
-  Compliance requirements (approval policies)
-  Quality control (brand safety)
-  Edge cases AI can't handle

**When to Skip HITL:**
-  High-volume, low-risk tasks
-  Well-defined, routine operations
-  Real-time requirements
-  Development/testing environments

### Production Patterns

```python
# Simple approval gate
class ApprovalGate(Executor):
    @handler
    async def review(self, content: str, ctx: WorkflowContext[str, str]):
        decision = input("Approve? (yes/no): ")
        if decision == 'yes':
            await ctx.yield_output(f"Approved: {content}")
        else:
            await ctx.yield_output("Rejected")

# Smart approval with threshold
class SmartGate(Executor):
    def __init__(self, threshold):
        super().__init__(id="smart_gate")
        self.threshold = threshold
    
    @handler
    async def review(self, data: dict, ctx: WorkflowContext):
        if data['risk_score'] < self.threshold:
            await ctx.yield_output("Auto-approved")
        else:
            decision = input("High risk - approve? ")
            await ctx.yield_output(decision)
```

### Real-World Applications

1. **Content Publishing Pipeline**
   - AI drafts content → Editor reviews → Publish if approved

2. **Customer Support Escalation**
   - AI handles routine queries → Escalate complex to human → Human provides guidance

3. **Financial Approval Workflow**
   - Auto-approve < $1000 → Manager approval $1000-$10k → Executive approval > $10k

4. **Data Access Control**
   - Auto-approve public data → Require approval for sensitive data

---

##  Practice Exercises

### Exercise 1: Multi-Level Approval

Create a workflow with escalating approvals:
```
Request → Manager ($0-$5k) → Director ($5k-$25k) → CFO (>$25k)
```

**Hint:** Use nested approval gates with different thresholds.

### Exercise 2: Feedback Loop with Memory

Create an interactive system that:
- Remembers previous feedback
- Shows improvement over iterations
- Learns user preferences

**Hint:** Maintain conversation history and reference it.

### Exercise 3: Approval Analytics

Track and report:
- Auto-approval rate
- Manual approval rate
- Average review time
- Rejection reasons

**Hint:** Add logging to approval gates.

In [ ]:
# Exercise playground - implement your solutions here!

async def multi_level_approval_exercise():
    """
    Exercise 1: Build escalating approval workflow.
    """
    # TODO: Implement multi-level approval
    pass

async def feedback_memory_exercise():
    """
    Exercise 2: Build feedback loop with memory.
    """
    # TODO: Implement feedback loop with context
    pass

async def approval_analytics_exercise():
    """
    Exercise 3: Track approval metrics.
    """
    # TODO: Implement analytics tracking
    pass

print(" Exercise templates ready - implement your solutions above!")

##  What's Next?

Congratulations! You've mastered Human-in-the-Loop patterns!

You now know how to:
-  Pause workflows for human approval
-  Create interactive refinement loops
-  Implement conditional branching based on decisions
-  Build smart approval gates with thresholds
-  Choose when to require human oversight

**Coming in Future Tutorials:**
- **Tutorial 09: Error Handling & Recovery** - Retry logic, fallbacks, graceful degradation
- **Tutorial 10: Monitoring & Observability** - Logging, metrics, tracing
- **Tutorial 11: Production Deployment** - Best practices, security, scaling

---

### Quick Reference

**Approval Gate Pattern:**
```python
class ApprovalGate(Executor):
    def __init__(self):
        super().__init__(id="approval")
    
    @handler
    async def review(self, content: str, ctx: WorkflowContext[str, str]):
        print(f"Review: {content}")
        decision = input("Approve? (yes/no): ")
        
        if decision == 'yes':
            await ctx.yield_output(f"Approved: {content}")
        else:
            await ctx.yield_output("Rejected")
```

**Interactive Loop Pattern:**
```python
conversation = [ChatMessage(role=Role.USER, content=request)]
for i in range(max_iterations):
    response = await agent.run(conversation)
    feedback = input("Feedback: ")
    conversation.append(ChatMessage(role=Role.ASSISTANT, content=response.text))
    conversation.append(ChatMessage(role=Role.USER, content=feedback))
```

**Smart Approval Pattern:**
```python
if risk_score < threshold:
    print("Auto-approved")
    await ctx.yield_output(content)
else:
    decision = input("High risk - approve? ")
    if decision == 'yes':
        await ctx.yield_output(content)
```

---

** Great job completing Tutorial 08!**